In [81]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import SMOTE
import string
import pickle
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/willcline/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [82]:
path = "../../data/News_Category_Dataset_v2.json"

In [83]:
df = pd.read_json(path,lines=True)

ValueError: Expected object or value

In [84]:
df.head(2)

,category,headline,authors,short_description,all_words,lower_desc
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,She left her husband. He killed their children...,There Were 2 Mass Shootings In Texas Last Week...,2 mass shootings texas last week 1 tv
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,Of course it has a song.,Will Smith Joins Diplo And Nicky Jam For The 2...,smith joins diplo nicky jam 2018 world cups of...


In [85]:
df['all_words'] = df['headline'] + "; " + df['short_description']

In [86]:
app_test = df.head(1)

In [87]:
app_test = df[df['category']=="POLITICS"].reset_index()

In [88]:
app_test['article'] = "test"

In [36]:
app_test = app_test[app_test["index"]==13]

In [37]:
ARTICLE = """
Tens of thousands of children from Central America are apprehended at the U.S.-Mexico border each year, straining the government system caring for minors traveling to the U.S. without their parents.
Now Donald Trump’s administration is stretching the system even further. 
Until recently, families that illegally crossed together generally faced deportation proceedings in civil court. But as of this month, the Trump administration is following a blanket policy of referring for prosecution all people who cross illegally.\
The change means that authorities send parents to jails run by the U.S. Marshals Services and their children wind up in the same agency as minors who came to the U.S. without their parents ― sometimes without their parents being able to locate them. 

White House chief of staff John Kelly told NPR this month that once separated from their parents, children “will be taken care of — put into foster care or whatever.” 

That “whatever” is the custody of the Office of Refugee Resettlement, part of the Department of Health and Human Services, which takes custody of unaccompanied children apprehended at the border, who by law may not be immediately deported
The ORR is charged with finding a sponsor for the children to live with ― ideally, a parent. 
Even before Trump administration officials announced the new policy, the ORR was showing signs of strain. 
While the number of children apprehended while traveling without their parents is lower than it was at its peak in 2014, border agents picked up more than 26,000 unaccompanied minors since October.
The ORR recently lost track of nearly 1,500 children once in its custody and in 2014 released some minors to traffickers. 

“They’re crippling an already overwhelmed system,” Michelle Brané, the director of migrant rights and justice at the Women’s Refugee Commission, said of the administration’s decision.
“You take that situation in which children are released, and then there’s little to no follow-up, and add hundreds and hundreds of children ― thousands of children, eventually ― and you’re multiplying the level of the problem.” 

Last month Steven Wagner, the acting assistant secretary of the Administration for Children and Families, which includes the ORR, told a Senate subcommittee that the federal government was unable to locate 1,475 children who had been released from its custody when it called to check in on them from October the end of 2017.
An additional 28 had run away, and 52 were living with someone other than their initial sponsor, he said. 
Once an unaccompanied minor is placed with a sponsor, “he or she ceases to be in the custody of the U.S. government, and all HHS-provided subsistence — food, shelter, clothing, health care and education — ends at that point, and the child becomes the responsibility of his or her parent, guardian or sponsor,” a spokesperson for the agency said in a statement. 
The ORR typically calls to check in with sponsors 30 days after releasing a child, but that’s it.
The children have little in the way of other help; the government does not provide immigrants with attorneys for removal proceedings, even if they’re children, leaving them to navigate a complicated legal process on their own.
This is as horrific a policy as I’ve seen in 25-plus years doing civil rights work.
The ORR has already come under scrutiny from some members of Congress, particularly after it was revealed that it released several minors in 2014 to sponsors who ended up being traffickers.
HHS and the Department of Homeland Security were supposed to have a joint plan for dealing with unaccompanied minors by February 2017 but failed to meet their deadline.
The agencies told the Senate’s permanent subcommittee on investigations last month that they would complete their plan by July 30, according to the office of Sen. Rob Portman (R-Ohio), who chairs the subcommittee and has pressed for quicker action. 

Now the government could be facing an influx of children, many of them likely to be younger than the average age of minors who traveled without parents.
Meanwhile, the Trump administration is taking steps that could deter some sponsors from coming forward to take in children.
The ORR previously checked fingerprints of potential sponsors to conduct background checks, and now it has a memorandum of understanding to share that information with Immigration and Customs Enforcement, which could have a chilling effect for undocumented parents already in the U.S. whose children came to join them. 

Parents in government custody are now in many cases unable to locate their children.
After the parents serve time for the misdemeanor offense of illegal entry ― typically a sentence of only a few days and often no more than time served ― they’re then transferred to ICE for deportation.

"""

In [38]:
app_test['article']= ARTICLE

In [75]:
app_test['link']

0    https://www.huffingtonpost.com/entry/immigrant...
Name: link, dtype: object

In [40]:
app_test.to_csv('app_test')

In [41]:
df['category'] = df['category'].replace(['CULTURE & ARTS'],'ARTS & CULTURE')

In [42]:
df.shape

(200853, 7)

In [43]:
df.drop(['link', 'date'], axis=1, inplace=True)

In [44]:
df.head(2)

,category,headline,authors,short_description,all_words
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,She left her husband. He killed their children...,There Were 2 Mass Shootings In Texas Last Week...
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,Of course it has a song.,Will Smith Joins Diplo And Nicky Jam For The 2...


### Make Target and features DFs

### Get list of stop words

In [47]:
# generate a list of stopwords for TfidfVectorizer to ignore
stopwords_list = stopwords.words('english') + list(string.punctuation)

### Basic cleaning

In [48]:
#lower case
df['lower_desc'] = df['headline'].apply(lambda x: " ".join(x.lower() for x in x.split()))

In [49]:
#remove punctuation
df['lower_desc'] = df['lower_desc'].str.replace('[^\w\s]','')

In [50]:
#stopword gathering and removal
stop = stopwords.words('english')
df['lower_desc'] = df['lower_desc'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

In [51]:
X = df.drop('category', axis=1)

In [52]:
y = df.category

### Vectorize data with tf-idf(one of the 3 vectorizer methods) and make a train test split

In [ ]:
#possibly unnecessary funciton to delete later on.
def preprocess_real_test(df):
    #requires stopwords list
    vectorizer = TfidfVectorizer(stop_words=stopwords_list)
    tf_idf = vectorizer.fit_transform(df)

In [26]:
# generate tf-idf vectorization (use sklearn's TfidfVectorizer) for our data
def tfidf(X, y,  stopwords_list): 
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
    vectorizer = TfidfVectorizer(stop_words=stopwords_list)
    tf_idf_train = vectorizer.fit_transform(X_train)
    tf_idf_test = vectorizer.transform(X_test)
    return tf_idf_train, tf_idf_test, y_train, y_test, vectorizer

In [27]:
#Call above function to vectorize and train test split data and put them in variables.
#Up here because presumably only have to do it once
idf_train, idf_test, y_tr, y_t, vectorizer = tfidf(X['all_words'], y, stopwords_list)

## Fit Smote Vectorized training data

In [77]:
sm = SMOTE(random_state=42)

In [79]:
sm.fit(idf_train, y_t)

NameError: name 'idf_train' is not defined

In [80]:
X_t_res, y_t_res = sm.sample(idf_train, y_t)

AttributeError: 'SMOTE' object has no attribute 'sample'

### Prediction function

In [29]:
def classify_text(classifier, tf_idf_train, tf_idf_test, y_train):
    classifier.fit(tf_idf_train, y_train)
    train_preds = classifier.predict(tf_idf_train)
    test_preds = classifier.predict(tf_idf_test)
    return train_preds, test_preds

### Scoring function to score predictions

In [30]:
def score_preds(y_test,y_train,test_preds, train_preds):
    print("Train: ", accuracy_score(y_train, train_preds))
    print("Test: ", accuracy_score(y_test, test_preds))
#     confusion_matrix(y_test, test_preds)
#     confusion_matrix(y_train, train_preds)

### Make baseline RF and NB models

In [31]:
rfc = RandomForestClassifier(n_estimators=100)
nb_classifier = MultinomialNB()

#### Pickle Real Test Vectorizer and Model

In [58]:
#create vectorizer object to fit to usable feature column of whole df
vectorizer = TfidfVectorizer(stop_words=stopwords_list)

In [59]:
vectorizer.fit(X['all_words'])

TfidfVectorizer(stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours',
                            'ourselves', 'you', "you're", "you've", "you'll",
                            "you'd", 'your', 'yours', 'yourself', 'yourselves',
                            'he', 'him', 'his', 'himself', 'she', "she's",
                            'her', 'hers', 'herself', 'it', "it's", 'its',
                            'itself', ...])

In [60]:
#pickle vectorizer
with open('vectorizer.pickle', 'wb') as handle:
     pickle.dump(vectorizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [61]:
#open vectorizer pickle
with open('vectorizer.pickle', 'rb') as handle:
    vectorizer = pickle.load(handle)

In [62]:
#Vectorize real test usable feature column
tf_idf_app_test = vectorizer.transform(app_test['all_words'])

In [63]:
tf_idf_app_test

<1x86526 sparse matrix of type '<class 'numpy.float64'>'
	with 24 stored elements in Compressed Sparse Row format>

In [64]:
#vectorize all of X to fit best model to for real predictions
tf_idf = vectorizer.transform(X['all_words'])

In [65]:
#fit best model to vectorized form of all rows of usable feature column. This would be the model used in deployment
rfc.fit(tf_idf, y)

RandomForestClassifier()

In [66]:
#pickle best model for category picker. Comment out afterwards
with open('genre_model.pickle', 'wb') as handle:
     pickle.dump(rfc, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [67]:
#open pickled, best model
with open('genre_model.pickle', 'rb') as handle:
    rfc = pickle.load(handle)

In [69]:
#make prediction with completed model on vectorized, usable column of real test dataframe
genre = rfc.predict(tf_idf_app_test)

In [74]:
genre[0]

array(['POLITICS'], dtype=object)

### Run Baseline Models with function calls

In [37]:
#Call function using results of vectorize function using NB model
nb_train_preds, nb_test_preds = classify_text(nb_classifier, idf_train, idf_test, y_tr)

In [38]:
#Call function for score
score_preds(y_t, y_tr, nb_test_preds, nb_train_preds)

Train:  0.4342899249198415
Test:  0.4021786752698451


In [ ]:
#Call function using results of vectorize function using RF model
rf_train_preds, rf_test_preds = classify_text(rfc, idf_train, idf_test, y_tr)

In [ ]:
#Call function for score
score_preds(y_t, y_tr, rf_test_preds, rf_train_preds)